# Trainen van een Groter Model

Het vorige model bleek te klein te zijn om de sinus deftig te voorspellen. Daarom trainen we nu een groter model.

## Configure Defaults

In [ ]:
# Define paths to model files
import os
MODELS_DIR = 'models/'
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
MODEL_TF = MODELS_DIR + 'model'
MODEL_NO_QUANT_TFLITE = MODELS_DIR + 'model_no_quant.tflite'
MODEL_TFLITE = MODELS_DIR + 'model.tflite'
MODEL_TFLITE_MICRO = MODELS_DIR + 'model.cc'

In [ ]:
# TensorFlow is an open source machine learning library
import tensorflow as tf

# Keras is TensorFlow's high-level API for deep learning
from tensorflow import keras
# Numpy is a math library
import numpy as np
# Pandas is a data manipulation library 
import pandas as pd
# Matplotlib is a graphing library
import matplotlib.pyplot as plt
# Math is Python's math library
import math

# Set seed for experiment reproducibility
seed = 1
np.random.seed(seed)
tf.random.set_seed(seed)

## Maken van een simpele dataset

We maken opnieuw een dataset met ruis en splitsen deze dan op in 3 delen.

In [ ]:
# Number of sample datapoints
SAMPLES = 1000

# Generate a uniformly distributed set of random numbers in the range from
# 0 to 2π, which covers a complete sine wave oscillation
x_values = np.random.uniform(
    low=0, high=2*math.pi, size=SAMPLES).astype(np.float32)

# Shuffle the values to guarantee they're not in order
np.random.shuffle(x_values)

# Calculate the corresponding sine values
y_values = np.sin(x_values).astype(np.float32)

# Add a small random number to each y value
y_values += 0.1 * np.random.randn(*y_values.shape)

# We'll use 60% of our data for training and 20% for testing. The remaining 20%
# will be used for validation. Calculate the indices of each section.
TRAIN_SPLIT =  int(0.6 * SAMPLES)
TEST_SPLIT = int(0.2 * SAMPLES + TRAIN_SPLIT)

# Use np.split to chop our data into three parts.
# The second argument to np.split is an array of indices where the data will be
# split. We provide two indices, so the data will be divided into three chunks.
x_train, x_test, x_validate = np.split(x_values, [TRAIN_SPLIT, TEST_SPLIT])
y_train, y_test, y_validate = np.split(y_values, [TRAIN_SPLIT, TEST_SPLIT])

# Double check that our splits add up correctly
assert (x_train.size + x_validate.size + x_test.size) ==  SAMPLES

# Plot the data in each partition in different colors:
plt.plot(x_train, y_train, 'b.', label="Training")
plt.plot(x_test, y_test, 'r.', label="Testing")
plt.plot(x_validate, y_validate, 'y.', label="Validation")
plt.legend()
plt.show()

## Trainen van een groter model

Door extra layers en neuronen toe te voegen kan je een model groter maken en krijgt het dus meer capaciteit om te leren.

### 1. Maken van een groter model

Om ons model groter te maken gaan we een extra layer toevoegen na de originele hidden layer. Ook gaan we beide layers uit 16 neuronen laten bestaan in plaats van 8.

TODO: Figuur van netwerk

Merk wel op dat dit er zal voor zorgen dat het trainingsproces meer tijd in beslag zal nemen en dat ons uiteindelijk model ook meer geheugen zal nodig hebben.

In [ ]:
model = tf.keras.Sequential(name="larger-model")

# First layer takes a scalar input and feeds it through 16 "neurons". The
# neurons decide whether to activate based on the 'relu' activation function.
model.add(keras.layers.Dense(16, activation='relu', input_shape=(1,), name="hidden-layer-0"))

# The new second and third layer will help the network learn more complex representations
model.add(keras.layers.Dense(16, activation='relu', name="hidden-layer-1"))

# Final layer is a single neuron, since we want to output a single value
model.add(keras.layers.Dense(1, name="output-layer"))

# Compile the model using the standard 'adam' optimizer and the mean squared error or 'mse' loss function for regression.
model.compile(optimizer='adam', loss="mse", metrics=["mae"])

# Output some information about the model
model.summary()

### 2. Trainen van het groter model

Laat ons vervolgens ook opnieuw dit model trainen.

**Opgelet! Als je onderstaande code om de een of andere reden nog eens zou uitvoeren, dan dien je bovenstaande code ook eerst opnieuw uit te voeren!**

In [ ]:
# Train the model
history = model.fit(x_train, y_train, epochs=500, batch_size=64,
                    validation_data=(x_validate, y_validate))

# Save the model to disk
model.save(MODEL_TF)

### 3. Grafiek van de statistieken

Elke training epoch drukt het model het verlies en de gemiddelde absolute fout af voor training en validatie. U kunt dit lezen in de uitvoer hierboven (merk op dat uw exacte cijfers kunnen verschillen):

```
Epoch 500/500
10/10 [==============================] - 0s 6ms/step - loss: 0.0117 - mae: 0.0870 - val_loss: 0.0111 - val_mae: 0.0848
```

Je kan zien dat we al een enorme verbetering hebben

- de `validation loss` is gedaald van `0,15` naar `0,01`
- de `validatie MAE` is gedaald van `0,33` naar `0,08`

De volgende cel zal dezelfde grafieken afdrukken die we gebruikten om ons oorspronkelijke model te evalueren, maar met onze nieuwe trainingsgeschiedenis

In [ ]:
# Draw a graph of the loss, which is the distance between
# the predicted and actual values during training and validation.
train_loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(train_loss) + 1)

# Exclude the first few epochs so the graph is easier to read
SKIP = 200

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)

plt.plot(epochs[SKIP:], train_loss[SKIP:], 'g.', label='Training loss')
plt.plot(epochs[SKIP:], val_loss[SKIP:], 'b.', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)

# Draw a graph of mean absolute error, which is another way of
# measuring the amount of error in the prediction.
train_mae = history.history['mae']
val_mae = history.history['val_mae']

plt.plot(epochs[SKIP:], train_mae[SKIP:], 'g.', label='Training MAE')
plt.plot(epochs[SKIP:], val_mae[SKIP:], 'b.', label='Validation MAE')
plt.title('Training and validation mean absolute error')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()

Dit zijn mooie resultaten. Uit deze grafieken kunnen we verschillende positieve dingen zien:

* De algemene `loss` en `MAE` zijn veel beter dan ons vorige netwerk
* Metrics zijn beter voor validatie dan voor training, wat betekent dat het netwerk niet overfitt is

De reden dat de statistieken voor validatie beter zijn dan die voor training, is dat validatiestatistieken worden berekend aan het einde van elke *epoch*, terwijl trainingsstatistieken gedurende de hele trainingsperiode worden berekend, dus validatie vindt plaats op een model dat iets langer is getraind.

Dit alles betekent dat ons netwerk goed lijkt te presteren. Laten we ter bevestiging de voorspellingen vergelijken met de test dataset die we eerder opzij hebben gezet:

In [ ]:
# Calculate and print the loss on our test dataset
test_loss, test_mae = model.evaluate(x_test, y_test)

# Make predictions based on our test dataset
y_test_pred = model.predict(x_test)

# Graph the predictions against the actual values
plt.clf()
plt.title('Comparison of predictions and actual values')
plt.plot(x_test, y_test, 'b.', label='Actual values')
plt.plot(x_test, y_test_pred, 'r.', label='TF predicted')
plt.legend()
plt.show()

Veel beter. De voorspellingen komen visueel vrij goed overeen met onze gegevens.

Het model is niet perfect. De voorspellingen vormen geen vloeiende sinuscurve. De lijn is bijvoorbeeld bijna recht als `x` tussen `2,5` en `4,5` ligt. Als we verder zouden willen gaan, zouden we kunnen proberen de capaciteit van het model verder te vergroten, misschien door enkele technieken te gebruiken om overfitting te voorkomen.

Een belangrijk onderdeel van machine learning is echter **weten wanneer te stoppen**. Dit model is goed genoeg voor onze use case - namelijk om sommige LED's in een aangenaam patroon te laten knipperen.

## Genereren van een TensorFlow Lite model

Onze volgende stappen bestaan eruit om een model te genereren dat we op de microcontroller kunnen runnen. Hiervoor moeten we ons TensorFlow model omvormen naar een TensorFlow Lite model. Natuurlijk moeten we het dan ook nog omzetten in code.

### 1. Genereren van een Model met en zonder kwantisatie

We hebben nu een acceptabel nauwkeurig model. We gebruiken de [TensorFlow Lite Converter](https://www.tensorflow.org/lite/convert) om het model om te zetten in een speciaal, ruimtebesparend formaat voor gebruik op devices met beperkte geheugencapaciteit.

Aangezien dit model op een microcontroller wordt ingezet, willen we dat het zo klein mogelijk is. Een techniek om de grootte van een model te verkleinen, wordt **[kwantisatie](https://www.tensorflow.org/lite/performance/post_training_quantization)** (quantization) genoemd. Het vermindert de precisie van de gewichten van het model, en mogelijk ook de activeringen (uitvoer van elke laag), wat geheugen bespaart, vaak zonder veel invloed op de nauwkeurigheid te hebben. Gekwantiseerde modellen werken ook sneller, omdat de vereiste berekeningen eenvoudiger zijn (gehele getallen versus komma getallen).

In de volgende sectie gaan we het model twee keer converteren: één keer met kwantisatie, één keer zonder.

In [ ]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_TF)
model_no_quant_tflite = converter.convert()

# Save the model to disk
open(MODEL_NO_QUANT_TFLITE, "wb").write(model_no_quant_tflite)

# Convert the model to the TensorFlow Lite format with quantization
def representative_dataset():
  for i in range(500):
    yield([x_train[i].reshape(1, 1)])
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce integer only quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_dataset
model_tflite = converter.convert()

# Save the model to disk
open(MODEL_TFLITE, "wb").write(model_tflite)

### 2. Vergelijking van de performantie van beide modellen

Om te bewijzen dat deze modellen nauwkeurig zijn, zelfs na conversie en kwantisatie, zullen we hun voorspellingen en verlies vergelijken met onze testdataset.

<!-- We define the `predict` (for predictions) and `evaluate` (for loss) functions for TFLite models. *Note: These are already included in a TF model, but not in  a TFLite model.* -->

In [ ]:
def predict_tflite(tflite_model, x_test):
  # Prepare the test data
  x_test_ = x_test.copy()
  x_test_ = x_test_.reshape((x_test.size, 1))
  x_test_ = x_test_.astype(np.float32)

  # Initialize the TFLite interpreter
  interpreter = tf.lite.Interpreter(model_content=tflite_model,
                                    experimental_op_resolver_type=tf.lite.experimental.OpResolverType.BUILTIN_REF)
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  # If required, quantize the input layer (from float to integer)
  input_scale, input_zero_point = input_details["quantization"]
  if (input_scale, input_zero_point) != (0.0, 0):
    x_test_ = x_test_ / input_scale + input_zero_point
    x_test_ = x_test_.astype(input_details["dtype"])
  
  # Invoke the interpreter
  y_pred = np.empty(x_test_.size, dtype=output_details["dtype"])
  for i in range(len(x_test_)):
    interpreter.set_tensor(input_details["index"], [x_test_[i]])
    interpreter.invoke()
    y_pred[i] = interpreter.get_tensor(output_details["index"])[0]
  
  # If required, dequantized the output layer (from integer to float)
  output_scale, output_zero_point = output_details["quantization"]
  if (output_scale, output_zero_point) != (0.0, 0):
    y_pred = y_pred.astype(np.float32)
    y_pred = (y_pred - output_zero_point) * output_scale

  return y_pred

def evaluate_tflite(tflite_model, x_test, y_true):
  global model
  y_pred = predict_tflite(tflite_model, x_test)
  loss_function = tf.keras.losses.get(model.loss)
  loss = loss_function(y_true, y_pred).numpy()
  return loss

**1. Voorspellingen**

In [ ]:
# Calculate predictions
y_test_pred_tf = model.predict(x_test)
y_test_pred_no_quant_tflite = predict_tflite(model_no_quant_tflite, x_test)
y_test_pred_tflite = predict_tflite(model_tflite, x_test)

# Compare predictions
plt.clf()
plt.title('Comparison of various models against actual values')
plt.plot(x_test, y_test, 'bo', label='Actual values')
plt.plot(x_test, y_test_pred_tf, 'ro', label='TF predictions')
plt.plot(x_test, y_test_pred_no_quant_tflite, 'bx', label='TFLite predictions')
plt.plot(x_test, y_test_pred_tflite, 'gx', label='TFLite quantized predictions')
plt.legend()
plt.show()

**2. Loss (MSE - Gemiddelde kwadratische fout)**

In [ ]:
# Calculate loss
loss_tf, _ = model.evaluate(x_test, y_test, verbose=0)
loss_no_quant_tflite = evaluate_tflite(model_no_quant_tflite, x_test, y_test)
loss_tflite = evaluate_tflite(model_tflite, x_test, y_test)

# Compare loss
df = pd.DataFrame.from_records(
    [["TensorFlow", loss_tf],
     ["TensorFlow Lite", loss_no_quant_tflite],
     ["TensorFlow Lite Quantized", loss_tflite]],
     columns = ["Model", "Loss/MSE"], index="Model").round(4)
df

**3. Grootte in geheugen**

In [ ]:
# Calculate size
size_tf = os.path.getsize(MODEL_TF)
size_no_quant_tflite = os.path.getsize(MODEL_NO_QUANT_TFLITE)
size_tflite = os.path.getsize(MODEL_TFLITE)

# Compare size
pd.DataFrame.from_records(
    [["TensorFlow", f"{size_tf} bytes", ""],
     ["TensorFlow Lite", f"{size_no_quant_tflite} bytes ", f"(reduced by {size_tf - size_no_quant_tflite} bytes)"],
     ["TensorFlow Lite Quantized", f"{size_tflite} bytes", f"(reduced by {size_no_quant_tflite - size_tflite} bytes)"]],
     columns = ["Model", "Size", ""], index="Model")

**Summary**

We kunnen aan de hand van de voorspellingen (grafiek) en de `loss` (tabel) zien dat het originele TF-model, het TFLite-model en het gekwantiseerde TFLite-model allemaal dicht genoeg bij elkaar liggen - ook al verschillen ze in grootte (tabel). Dit impliceert dat het gekwantiseerde (kleinste) model klaar is voor gebruik.

*Opmerking: het gekwantiseerde (integer) TFLite-model is slechts 500 bytes kleiner dan het originele (float) TFLite-model - een kleine verkleining! Dit komt doordat het model al zo klein is dat kwantisering minder effect heeft. Complexe modellen met meer gewicht kunnen tot 4x kleiner worden!*

## Genereren van een TensorFlow Lite model voor microcontrollers

Hier converteren we het gekwantiseerde model van TensorFlow Lite naar een C-bronbestand dat kan worden geladen door TensorFlow Lite voor microcontrollers.

In [ ]:
# Install xxd if it is not available
# !apt update && apt -qq install xxd

# Convert to a C source file, i.e, a TensorFlow Lite for Microcontrollers model
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}

# Update variable names
REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')
!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_TFLITE_MICRO}

# Print the C source file
!cat {MODEL_TFLITE_MICRO}

## Implementeren op een microcontroller

Je kan nu de instructies volgen in de slides om het model te implementeren in de bijhorende demo applicatie. Zo kan het model worden uitgevoerd op de microcontroller.